In [ ]:
import json
import base64
import gzip
import os
import shutil
import copy
from pathlib import Path
from typing import Union
from collections import defaultdict

In [ ]:
DRAW_COMMANDS = [
    "drawArrays",
    "drawElements",
    "drawArraysInstanced",
    "drawArraysInstancedANGLE",
    "drawElementsInstanced",
    "drawElementsInstancedANGLE",
    "drawRangeElements",
    "multiDrawArraysWEBGL",
    "multiDrawElementsWEBGL",
    "multiDrawArraysInstancedWEBGL",
    "multiDrawElementsInstancedWEBGL",
    "multiDrawArraysInstancedBaseInstanceWEBGL",
    "multiDrawElementsInstancedBaseVertexBaseInstanceWEBGL",
]

STATES = [
    "BlendState",
    "ColorState",
    "CoverageState",
    "CullState",
    "DepthState",
    "DrawState",
    "PolygonOffsetState",
    "ScissorState",
    "StencilState",
    "VisualState",
    "DrawCall"
]

PURGE_KEYS = [
    'value',
    'displayText',
]

In [ ]:
def purge(co: Union[dict,list,int,str,float,bool,None]):
    if co is None:
        return None
    if isinstance(co, (int, str, float, bool)):
        return co
    if isinstance(co, list):
        return [purge(x) for x in co]
    if isinstance(co, dict):
        ret = {k: purge(v) for k, v in co.items() if k not in PURGE_KEYS and not k.startswith("__SPECTOR") and 'CommandId' not in k}
    return ret

In [ ]:
# obj = json.load(gzip.open('output/TEST-1_0_0.json.gz', 'rt'))

for p in sorted(Path('output').glob('*.json.gz')):
    obj = json.load(gzip.open(p, 'rt'))

    possible_pipelines_count = defaultdict(int)
    possible_pipelines_set = defaultdict(set)

    for cmd in obj['commands']:
        if cmd['name'] in DRAW_COMMANDS:
            vv = purge({k: cmd[k] for k in STATES})
            possible_pipelines_count[cmd['name']] += 1
            possible_pipelines_set[cmd['name']].add(json.dumps(vv, sort_keys=True, separators=(",", ":"), indent=None))
    possible_pipelines_count_dedup = {k: len(v) for k, v in possible_pipelines_set.items()}
    print(p)
    print("Before de-dup:", json.dumps(possible_pipelines_count, sort_keys=True, separators=(",", ":"), indent=None))
    print(" After de-dup:", json.dumps(possible_pipelines_count_dedup, sort_keys=True, separators=(",", ":"), indent=None))